# Exploratory Data Analysis for New York Taxi Dataset
1. Read nyc 2022 data
2. Organize it into number of arrivals and number of departures by location for each hour format
3. Organize it into a tensor
4. Generate similarity graph for locations
5. Run LR-STSS on it
6. Visualize it on a map 

Make an interactive dashboard with it

In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
# import earthpy as et
import networkx as nx
from tqdm import tqdm
from scipy import io

import project_path
from sklearn.neighbors import kneighbors_graph, radius_neighbors_graph
from src.util.t2m import t2m
from src.algos.lr_stss import lr_stss
from dask.distributed import Client, progress, wait
from src.algos.lr_stss_modified import lr_stss_modified
from IPython.display import display, HTML

### Load Data

In [6]:
# Load pre-processed Data
cwd = os.getcwd()
data_dir = os.path.join(cwd,'..','data','nyc')

zones = gpd.read_file(os.path.join(data_dir,'taxi_zones_shapefile','taxi_zones.shp'))
zone_lookup = os.path.join(data_dir, 'taxi_zone_lookup.csv')
zone_lookup = pd.read_csv(zone_lookup)

pickups = np.load(os.path.join(data_dir,"hourly_pickup.npy"))
dropoffs = np.load(os.path.join(data_dir,"hourly_dropoff.npy"))
# # Load Emre's settings
dates = io.loadmat(os.path.join(data_dir,'dates.mat'))
regions = io.loadmat(os.path.join(data_dir,'regions.mat'))
neighbors = io.loadmat(os.path.join(data_dir,'neighbors.mat'))
regions=regions['regions'].ravel()

# # Filter the data
pickups = pickups[regions-1, ...]
dropoffs = dropoffs[regions-1, ...]
zones = zones.iloc[regions-1]
dos = np.zeros((81,53,7,24))
dos[:,:52,:,:] = dropoffs
dos[:,52,1:,:] = np.mean(dropoffs[:,:,1:,:],1)
dropoffs = dos.copy()
dos[:,:52,:,:] = pickups
dos[:,52,1:,:] = np.mean(pickups[:,:,1:,:],1)
pickups = dos.copy()
del dos

## Create Graph

In [7]:
pos = np.zeros((81,2))
pos[:,0] = zones.geometry.centroid.x.values
pos[:,1] = zones.geometry.centroid.y.values

position ={}
G_nyc = nx.Graph()
G_nyc.add_nodes_from([(regions[i], {'pos': pos[i,:], 'LocationID': regions[i], 'zone': zones.iloc[i]['zone']}) for i in range(81)])
edge_list =[]
for i in range(len(neighbors['neighbors'].ravel())):
    for neighbor in neighbors['neighbors'].ravel()[i].ravel():
        if np.any(np.isin(neighbor, regions)) and (neighbor!=regions[i]):
            edge_list.append((regions[i], neighbor))
    # edge_list = edge_list + [(regions[i], neighbor) for neighbor in neighbors['neighbors'].ravel()[i]]
    position[regions[i]]=pos[i,:]
G_nyc.add_edges_from(edge_list)
# G_nyc.nodes()
A = nx.adjacency_matrix(G_nyc).toarray()
Deg = np.diag(np.asarray(np.sum(A,axis=1)).ravel())
Dsq = np.linalg.inv(np.sqrt(Deg))
An = Dsq@A@Dsq

In [8]:
psi_params = np.sqrt(np.array([np.trace(np.cov(t2m(dropoffs,i))) for i in [1,2,3,4]]))
psi_dist = psi_params/np.max(psi_params)
print(f"psi distribution:\t {psi_dist}")
print("Tensor dimension:\t",dropoffs.shape)

psi distribution:	 [0.83150206 1.         0.36393466 0.63659289]
Tensor dimension:	 (81, 53, 7, 24)


$\psi = [0.65499501, 1.        , 0.13491694, 0.42404973] $

Later used: $\psi = [0.80931762, 1.        , 0.36731042, 0.65119101] $

Pickups: $\psi = [0.65499501, 1.        , 0.13491694, 0.42404973] $

Dropoffs: $\psi = [0.83768957, 1.        , 0.36744661, 0.64250066]$

Tensor dimension:	 $(81, 52, 7, 24)$

In [19]:
time_m = 4
local_m = 1
lda_2 = 1000
psi = 20
lda_1 = 1
lda_t = 0.5
lda_loc = 0.5
res = lr_stss_modified(dropoffs, An, time_m,local_m, verbose=1, max_it=150,
    lda2=lda_2, lda1=lda_1, lda_t=lda_t,
    lda_loc=lda_loc, psis=psi_dist*psi, rho=0.0005, rho_upd=-1)
abs_s = np.abs(res['S'])
ratios = np.array([0.014, 0.07, 0.14, 0.3, 0.7, 1, 2, 3])/100
num_detected_events = [detect_topk_events(abs_s, r) for r in ratios]
print(num_detected_events)


It-2:	## |r|=46255.56542 	 ## |s|=160816.14209 	 ## rho=0.0005 obj=12177308.9630 	 ## del_obj = 3688097.6113 
It-3:	## |r|=41919.47891 	 ## |s|=30207.11498 	 ## rho=0.0005 obj=12702176.1063 	 ## del_obj = 524867.1434 
It-4:	## |r|=36110.51987 	 ## |s|=30710.98413 	 ## rho=0.0005 obj=13353023.0383 	 ## del_obj = 650846.9320 
It-5:	## |r|=31479.43731 	 ## |s|=24305.27303 	 ## rho=0.0005 obj=14000843.6734 	 ## del_obj = 647820.6351 
It-6:	## |r|=28523.69495 	 ## |s|=19091.27397 	 ## rho=0.0005 obj=14506832.5809 	 ## del_obj = 505988.9075 
It-7:	## |r|=26628.76600 	 ## |s|=12384.02395 	 ## rho=0.0005 obj=14819695.7299 	 ## del_obj = 312863.1490 
It-8:	## |r|=24481.22161 	 ## |s|=15924.25409 	 ## rho=0.0005 obj=15239481.7524 	 ## del_obj = 419786.0225 
It-9:	## |r|=22496.12336 	 ## |s|=13468.47300 	 ## rho=0.0005 obj=15594967.4352 	 ## del_obj = 355485.6827 
It-10:	## |r|=21171.00423 	 ## |s|=10273.90551 	 ## rho=0.0005 obj=15876108.0292 	 ## del_obj = 281140.5940 
It-11:	## |r|=19746.08934

In [18]:
io.savemat('nyc_taxi_lrstss_byhand_1.mat', {'S':np.moveaxis(res['S'],[0,1,2,3],[3,2,1,0])})
# res = io.loadmat('nyc_taxi_run.mat')


## Anomaly Scoring of the Points

In [11]:
# def detect_topk_events(anomaly_scores, ratio):
#     events_start_ts = pd.to_datetime(['01-Jan-2018', '03-Jan-2018 16:00:00', '14-Jan-2018 09:00:00', '20-Jan-2018 08:00:00', 
#                                     '28-Jan-2018 16:00:00', '04-Mar-2018 15:00:00', '31-Mar-2018 13:00:00', '17-Mar-2018 11:00:00',
#                                     '20-Mar-2018 10:00:00', '21-Mar-2018 16:00:00', '01-Jul-2018 17:00:00', '04-Jul-2018 17:00:00',
#                                     '25-Sep-2018 10:00:00', '04-Oct-2018 08:00:00', '04-Nov-2018 12:00:00', '09-Nov-2018 19:00:00',
#                                     '22-Nov-2018 21:00:00', '04-Dec-2018 19:00:00', '16-Dec-2018 10:00:00', '31-Dec-2018 21:00:00'
#                                     ])

#     events_end_ts = pd.to_datetime(['01-Jan-2018 02:00:00', '03-Jan-2018 22:00:00', '14-Jan-2018 17:00:00', '20-Jan-2018 15:00:00',
#                                 '28-Jan-2018 23:00:00', '04-Mar-2018 22:00:00', '31-Mar-2018 20:00:00', '17-Mar-2018 17:00:00',
#                                 '20-Mar-2018 20:00:00', '21-Mar-2018 22:00:00', '01-Jul-2018 22:00:00', '04-Jul-2018 23:00:00',
#                                 '25-Sep-2018 20:00:00', '04-Oct-2018 15:00:00', '04-Nov-2018 17:00:00', '09-Nov-2018 23:30:00',
#                                 '22-Nov-2018 23:59:00', '04-Dec-2018 23:59:00', '16-Dec-2018 15:00:00', '31-Dec-2018 23:59:00'
#     ])
#     indd = np.flip(np.argsort(anomaly_scores, axis=None))
#     ind = np.unravel_index(indd[:int(len(indd)*ratio)], anomaly_scores.shape)
#     topk_event_idx = ind
#     anomaly_mask = np.zeros(anomaly_scores.shape, dtype=bool)
#     anomaly_mask[topk_event_idx] =1
#     num_detected_events = 0


#     idxs = np.arange(81)
#     # w = events_start_ts.isocalendar().week
#     # d = events_start_ts.day_of_week
#     doy = events_start_ts.day_of_year
#     w = (doy-1)//(7)
#     d = (doy-1) % 7
#     h_s = events_start_ts.hour
#     h_e = events_end_ts.hour
#     for i in range(20):
#         event_mask = np.zeros(anomaly_scores.shape, dtype=bool)
#         locations = dates['dates'][2].ravel()[i].ravel()
        
#         for loc in locations: 
#             # event_mask[idxs[regions==loc], w[i]-1, d[i], h_s[i]:h_e[i]] = 1
#             # event_mask[idxs[regions==loc], w[i]-1, d[i], h_e[i]] = 1
#             event_mask[idxs[regions==loc], w[i], d[i], h_s[i]:h_e[i]] = 1
#             event_mask[idxs[regions==loc], w[i], d[i], h_e[i]] = 1
#         if np.any(event_mask & anomaly_mask):
#             num_detected_events +=1
#     return num_detected_events

def detect_topk_events(anomaly_scores, ratio):
#     events_start_ts = pd.to_datetime(['01-Jan-2018', '03-Jan-2018 16:00:00', '14-Jan-2018 09:00:00', '20-Jan-2018 08:00:00', 
#                                     '28-Jan-2018 16:00:00', '04-Mar-2018 15:00:00', '31-Mar-2018 13:00:00', '17-Mar-2018 11:00:00',
#                                     '20-Mar-2018 10:00:00', '21-Mar-2018 16:00:00', '01-Jul-2018 17:00:00', '04-Jul-2018 17:00:00',
#                                     '25-Sep-2018 10:00:00', '04-Oct-2018 08:00:00', '04-Nov-2018 12:00:00', '09-Nov-2018 19:00:00',
#                                     '22-Nov-2018 21:00:00', '04-Dec-2018 19:00:00', '16-Dec-2018 10:00:00', '31-Dec-2018 21:00:00'
#                                     ])
    events_start_ts = pd.to_datetime(['01-Jan-2018', '03-Jan-2018 16:00:00', '14-Jan-2018 09:00:00', '20-Jan-2018 08:00:00', 
                                    '4-Mar-2018 15:00:00', '08-Mar-2018 18:00:00', '17-Mar-2018 11:00:00', '20-Mar-2018 10:00:00',
                                    '21-Mar-2018 16:00:00', '01-Jul-2018 17:00:00', '04-Jul-2018 17:00:00', '25-Sep-2018 10:00:00',
                                    '04-Oct-2018 08:00:00', '04-Nov-2018 12:00:00', '09-Nov-2018 19:00:00', '22-Nov-2018 21:00:00',
                                    '4-Dec-2018 19:00:00', '16-Dec-2018 10:00:00', '28-Dec-2018 12:00:00', '31-Dec-2018 21:00:00'
    ])
#     events_end_ts = pd.to_datetime(['01-Jan-2018 02:00:00', '03-Jan-2018 22:00:00', '14-Jan-2018 17:00:00', '20-Jan-2018 15:00:00',
#                                 '28-Jan-2018 23:00:00', '04-Mar-2018 22:00:00', '31-Mar-2018 20:00:00', '17-Mar-2018 17:00:00',
#                                 '20-Mar-2018 20:00:00', '21-Mar-2018 22:00:00', '01-Jul-2018 22:00:00', '04-Jul-2018 23:00:00',
#                                 '25-Sep-2018 20:00:00', '04-Oct-2018 15:00:00', '04-Nov-2018 17:00:00', '09-Nov-2018 23:30:00',
#                                 '22-Nov-2018 23:59:00', '04-Dec-2018 23:59:00', '16-Dec-2018 15:00:00', '31-Dec-2018 23:59:00'
#     ])
    events_end_ts = pd.to_datetime(['01-Jan-2018 02:00:00', '03-Jan-2018 22:00:00', '14-Jan-2018 17:00:00', '20-Jan-2018 15:00:00',
                             '4-Mar-2018 22:00:00', '08-Mar-2018 23:59:00', '17-Mar-2018 17:00:00', '20-Mar-2018 20:00:00',
                            '21-Mar-2018 22:00:00', '01-Jul-2018 22:00:00', '04-Jul-2018 23:00:00', '25-Sep-2018 20:00:00',
                            '04-Oct-2018 15:00:00', '04-Nov-2018 17:00:00', '09-Nov-2018 23:30:00', '22-Nov-2018 23:59:00',
                            '4-Dec-2018 23:59:00', '16-Dec-2018 15:00:00', '28-Dec-2018 17:00:00', '31-Dec-2018 23:59:00'
    ])
    indd = np.flip(np.argsort(anomaly_scores, axis=None))
    ind = np.unravel_index(indd[:int(len(indd)*ratio)], anomaly_scores.shape)
    topk_event_idx = ind
    anomaly_mask = np.zeros(anomaly_scores.shape, dtype=bool)
    anomaly_mask[topk_event_idx] =1
#     num_detected_events = 0
    detected_events = np.zeros(20)

    idxs = np.arange(81)
    # w = events_start_ts.isocalendar().week
    # d = events_start_ts.day_of_week
    doy = events_start_ts.day_of_year
    w = (doy-1)//(7)
    d = (doy-1) % 7
    h_s = events_start_ts.hour
    h_e = events_end_ts.hour
    for i in range(20):
        event_mask = np.zeros(anomaly_scores.shape, dtype=bool)
        locations = dates['dates'][2].ravel()[i].ravel()
        
        for loc in locations: 
            # event_mask[idxs[regions==loc], w[i]-1, d[i], h_s[i]:h_e[i]] = 1
            # event_mask[idxs[regions==loc], w[i]-1, d[i], h_e[i]] = 1
            event_mask[idxs[regions==loc], w[i], d[i], h_s[i]:h_e[i]] = 1
            event_mask[idxs[regions==loc], w[i], d[i], h_e[i]] = 1
        if np.any(event_mask * anomaly_mask):
#             num_detected_events +=1
            detected_events[i]=1
    return detected_events#num_detected_events

In [5]:
client = Client(n_workers=125)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 125
Total threads: 250,Total memory: 100.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43014,Workers: 125
Dashboard: http://127.0.0.1:8787/status,Total threads: 250
Started: Just now,Total memory: 100.00 GiB
Comm: tcp://127.0.0.1:41624,Total threads: 2
Dashboard: http://127.0.0.1:36198/status,Memory: 819.20 MiB
Nanny: tcp://127.0.0.1:42195,


In [6]:
## Control Variables
time_m = 4
local_m = 1
maxit = 100
psi_dist = np.array([0.83768957, 1.        , 0.36744661, 0.64250066])
# psi_dist = np.array([0.65499501, 1, 0.13491694, 0.42404973])
# psi_dist = np.array([0.80931762, 1.        , 0.36731042, 0.65119101])
## Independent variables
### Hyperparameters
rhos = [0.0005, 0.00075, 0.001, 0.002, 0.01]
lda_2 = [100.0]
psi = [32.0, 25.0, 5.0, 60.0] #np.linspace(47,30,4)
lda_1 = list(np.linspace(120,1,6))+[0.5,0.1] # np.linspace(15,60,6)# np.logspace( np.log10(110),1.5,4)
lda_l =  np.logspace(2.5,-1.5,11) # np.linspace(20,1, 6)
lda_t =  np.logspace(2.5,-1.5,11) # np.array(list(np.linspace(100,1,4))+[0.5, 0.1]) # [100]

Deg = np.diag(np.asarray(np.sum(A,axis=1)).ravel())
Dsq = np.linalg.inv(np.sqrt(Deg))
An = Dsq@A@Dsq
# Y = np.ma.masked_array(dropoffs, mask=np.zeros(pickups.shape,dtype=bool))
Y = dropoffs
def run_exp(inputs):
    Y_ = Y.copy()
    An_= An.copy()
    psi = inputs['psi']
    res = lr_stss_modified(Y_, An_, time_m,local_m, verbose=0, max_it=maxit,
        lda2=inputs['lda_2'], lda1=inputs['lda_1'], lda_t=inputs['lda_t'],
        lda_loc=inputs['lda_l'], psis=psi_dist*psi, rho=inputs['rho'], rho_upd=-1)
    abs_s = np.abs(res['S'])
    ratios = np.array([0.014, 0.07, 0.14, 0.3, 0.7, 1, 2, 3])/100
    num_detected_events = np.array([detect_topk_events(abs_s, r) for r in ratios])
    result = {'topk_detection_sum': np.sum(num_detected_events),
              '0.014': num_detected_events[0],
              '0.07': num_detected_events[1],
              '0.14': num_detected_events[2],
              '0.3': num_detected_events[3],
              '0.7': num_detected_events[4],
              '1': num_detected_events[5],
              '2':num_detected_events[6],
              '3':num_detected_events[7],
              'lda_1': inputs['lda_1'],
              'lda_2': inputs['lda_2'],
              'lda_l': inputs['lda_l'],
              'lda_t': inputs['lda_t'],
              'psi': inputs['psi'],
              'maxit': maxit,
              'rho': inputs['rho'],
              'it': res['it']}
    return result

In [ ]:
fname = 'nyc_lr_stss_hyperparameter_study_final.hdf5'
key = 'dropoff'
cwd = os.getcwd()
fname = os.path.join(cwd, 'results', fname)
try:
    del futures
except:
    pass
finally:
    pass
futures = []
inputs = {}
idx=0
for r1 in range(len(rhos)):
    for k4 in range(len(psi)):
        for k1 in range(len(lda_1)):
            for k2 in range(len(lda_l)):
                for k3 in range(len(lda_t)):
                    for l2 in lda_2:
                        inputs['rho']=rhos[r1]
                        inputs['lda_1'] = lda_1[k1]
                        inputs['lda_l'] = lda_l[k2]
                        inputs['lda_t'] = lda_t[k3]
                        inputs['lda_2'] = l2
                        inputs['psi'] = psi[k4]
                        futures.append(client.submit(run_exp, inputs))
        
            wait(futures)
            for i in range(len(futures)):
                df = pd.DataFrame(futures[i].result(), index=[idx])
                df = df.astype({'psi':'int64'})
                with pd.HDFStore(fname) as storedata:
                    storedata.append(key, df)
                idx+=1
            del futures
            futures = []

In [26]:
key = 'dropoff2'
for i in range(len(futures)):
    df = pd.DataFrame(futures[i].result(), index=[idx])
    df = df.astype({'psi':'int64'})
    with pd.HDFStore(fname) as storedata:
        storedata.append(key, df)
    idx+=1
del futures
futures = []

In [ ]:
progress(futures)

### Save results
fname = 'nyc_lr_stss_hyperparameter_study.hdf5'

pickup keys: 'grid#' [1-6]

dropoff keys: 'dropoff#'

In [12]:
fname = 'nyc_lr_stss_hyperparameter_study.hdf5'
key = 'dropoff4'
cwd = os.getcwd()
fname = os.path.join(cwd, 'results', fname)

for i in range(len(futures)):
    df = pd.DataFrame(futures[i].result(), index=[i])
    with pd.HDFStore(fname) as storedata:
        storedata.append(key, df)

# with pd.HDFStore(fname) as storedata:
#     storedata.get_storer(key).attrs.metadata=metadata

ValueError: cannot match existing table structure for [lda_1,lda_2,psi] on appending data

### Load Results


In [8]:
fname = 'nyc_lr_stss_hyperparameter_study_final.hdf5'
key = 'dropoff'
cwd = os.getcwd()
fname = os.path.join(cwd, 'results', fname)
with pd.HDFStore(fname) as storedata:
    df = storedata[key]
    # metadata = storedata.get_storer(key).attrs.metadata
display(HTML(df.sort_values(['3','topk_detection_sum'], ascending=False).to_html()))
# df.info()
# print("Metadata:\n",metadata)

,topk_detection_sum,0.014,0.07,0.14,0.3,0.7,1,2,3,lda_1,lda_2,lda_l,lda_t,psi,maxit,rho,it
0,77,3,4,5,8,13,13,15,16,45.0,100.0,100.0,100.0,32,100,0.0005,100
1,77,3,4,5,8,13,13,15,16,45.0,100.0,100.0,100.0,32,100,0.0010,100
2,77,3,4,5,8,13,13,15,16,45.0,100.0,100.0,100.0,32,100,0.0020,100
4,77,3,4,5,8,13,13,15,16,45.0,100.0,100.0,100.0,32,100,0.0010,100
6,77,3,4,5,8,13,13,15,16,45.0,100.0,100.0,67.0,32,100,0.0005,100
7,77,3,4,5,8,13,13,15,16,45.0,100.0,100.0,67.0,32,100,0.0010,100
8,77,3,4,5,8,13,13,15,16,45.0,100.0,100.0,67.0,32,100,0.0020,100
10,77,3,4,5,8,13,13,15,16,45.0,100.0,100.0,67.0,32,100,0.0010,100
12,77,3,4,5,8,13,13,15,16,45.0,100.0,100.0,34.0,32,100,0.0005,100
13,77,3,4,5,8,13,13,15,16,45.0,100.0,100.0,34.0,32,100,0.0010,100


2023-08-09 00:07:56,998 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 559.64 MiB -- Worker memory limit: 806.30 MiB
2023-08-09 00:08:07,196 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 558.04 MiB -- Worker memory limit: 806.30 MiB
2023-08-09 00:08:15,894 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more informati

In [14]:
import plotly.express as px

In [15]:
fig = px.parallel_coordinates(df, color="topk_detection_sum",
                              dimensions=['lda_2', 'psi', 'lda_1',
                                          'lda_l', 'lda_t', '3','topk_detection_sum', '2'], #'0.014','0.07','0.14','0.3',
                              color_continuous_scale=px.colors.diverging.Tealrose)
fig.show()

/mnt/home/indibimu/miniconda3/lib/python3.9/site-packages/plotly/express/_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [10]:
client.shutdown()

2023-08-09 00:10:21,235 - distributed.nanny - WARNING - Worker process still alive after 3.199999389648438 seconds, killing
2023-08-09 00:10:21,236 - distributed.nanny - WARNING - Worker process still alive after 3.1999986267089846 seconds, killing


In [8]:
## Control Variables
time_m = 4
local_m = 1
lda_2 = 100
psi = 10
lda_1 = 31.622777
lda_t = 2.154435
lda_loc = 3.981072
maxit = 80
maxit2 = 60
psi_dist = np.array([0.83768957, 1.        , 0.36744661, 0.64250066])
# lda_1	lda_2	lda_l	lda_t	psi
# 31.622777	100.0	3.981072	2.154435	10
# time_m = 4, local_m = 1, lda_2 = 100, psi = 20, lda_1 = 1, lda_t = 0.5, lda_loc = 0.1, maxit = 300, maxit2 = 60
# 90.000000	100.0	10.000000	20.000000	25.0	
Deg = np.diag(np.asarray(np.sum(A,axis=1)).ravel())
Dsq = np.linalg.inv(np.sqrt(Deg))
An = Dsq@A@Dsq
# Run 1 lda_2=100, psi=20, lda_1=1, lda_t=0.5, lda_loc=0.5
# Run 2
Y = np.ma.masked_array(dropoffs, mask=np.zeros(pickups.shape,dtype=bool))
res = lr_stss(Y, An, time_m,local_m, verbose=1, max_it2=maxit2, max_it=maxit,
        lda2=lda_2, lda1=lda_1, lda_t=lda_t,
        lda_loc=lda_loc, psis=psi_dist*psi, rho=1, rho_upd=-1) # [10*psi,psi,4*psi,3*psi])
abs_s = np.abs(res['S'])
ratios = np.array([0.014, 0.07, 0.14, 0.3, 0.7, 1, 2, 3])/100
num_detected_events = np.array([detect_topk_events(abs_s, r) for r in ratios])
print(num_detected_events)

It-2:	## |r|=5462.97745 	 ## |s|=14735.99263 	 ## rho=1.0000 obj=124659031.1219 	 ## del_obj = -1777177.7739 
It-3:	## |r|=3322.56063 	 ## |s|=20708.87397 	 ## rho=1.0000 obj=122292157.0080 	 ## del_obj = -2366874.1139 
It-4:	## |r|=2276.27503 	 ## |s|=24543.07987 	 ## rho=1.0000 obj=119449929.5557 	 ## del_obj = -2842227.4523 
It-5:	## |r|=1654.07817 	 ## |s|=27168.30669 	 ## rho=1.0000 obj=116288635.3544 	 ## del_obj = -3161294.2013 
It-6:	## |r|=1260.54470 	 ## |s|=29063.77680 	 ## rho=1.0000 obj=112901255.0792 	 ## del_obj = -3387380.2751 
It-7:	## |r|=998.23079 	 ## |s|=30498.26971 	 ## rho=1.0000 obj=109347998.6009 	 ## del_obj = -3553256.4783 
It-8:	## |r|=807.89923 	 ## |s|=31607.38763 	 ## rho=1.0000 obj=105671776.2231 	 ## del_obj = -3676222.3779 
It-9:	## |r|=664.06521 	 ## |s|=32477.34102 	 ## rho=1.0000 obj=101905572.9073 	 ## del_obj = -3766203.3158 
It-10:	## |r|=550.95805 	 ## |s|=33158.63109 	 ## rho=1.0000 obj=98077772.1158 	 ## del_obj = -3827800.7915 
It-11:	## |r|=

/tmp/local/20223090/ipykernel_55458/1261714271.py:22: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  w = events_start_ts.week
/tmp/local/20223090/ipykernel_55458/1261714271.py:22: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  w = events_start_ts.week
/tmp/local/20223090/ipykernel_55458/1261714271.py:22: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().w

[ 0  0  0  0  3  3  9 10]


/tmp/local/20223090/ipykernel_55458/1261714271.py:22: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  w = events_start_ts.week
/tmp/local/20223090/ipykernel_55458/1261714271.py:22: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  w = events_start_ts.week


## Anomaly Scoring

In [11]:
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from src.util.t2m import t2m
from src.util.m2t import m2t
from sklearn.covariance import EllipticEnvelope

def ee_anomaly_score(s):
    anomaly_scores = np.zeros(s.shape)
    for l in range(s.shape[0]):
        for d in range(s.shape[2]):
            for h in range(s.shape[3]):
                features = s[l,:,d,h].reshape((52,1))
                model = EllipticEnvelope(random_state=0, contamination=0.05)
                model.fit(features)
                anomaly_scores[l,:,d,h] = model.mahalanobis(features)
    return anomaly_scores

def lof_anomaly_score(s):
    anomaly_scores = np.zeros(s.shape)
    for l in range(s.shape[0]):
        for d in range(s.shape[2]):
            for h in range(s.shape[3]):
                features = s[l,:,d,h].reshape((52,1))
                model = LocalOutlierFactor(algorithm='kd_tree',n_neighbors=5, contamination=0.1, n_jobs=5)
                model.fit(features)
                anomaly_scores[l,:,d,h] = -model.negative_outlier_factor_
    return anomaly_scores

def ocsvm_anomaly_score(s):
    anomaly_scores = np.zeros(s.shape)
    for l in range(s.shape[0]):
        for d in range(s.shape[2]):
            for h in range(s.shape[3]):
                features = s[l,:,d,h].reshape((52,1))
                model = OneClassSVM()
                model.fit(features)
                anomaly_scores[l,:,d,h] = model.score_samples(features)
    return anomaly_scores

In [13]:
ratios = np.array([0.014, 0.07, 0.14, 0.3, 0.7, 1, 2, 3])/100
abs_s = np.abs(res['S'])
ee_s = ee_anomaly_score(res['S'])
ee_abs_s = ee_anomaly_score(abs_s)
# lof_s = lof_anomaly_score(abs_s)
# ocsvm_s = ocsvm_anomaly_score(res['S'])

# num_detected_events_vanilla = [detect_topk_events(abs_s, r) for r in ratios]
num_detected_events_ee = [detect_topk_events(ee_s, r) for r in ratios]
# num_detected_events_ee_abs_s = [detect_topk_events(ee_abs_s, r) for r in ratios]
# num_detected_events_lof = [detect_topk_events(lof_s, r) for r in ratios]
# num_detected_events_ocsmv = [detect_topk_events(ocsvm_s, r) for r in ratios]

print(num_detected_events_vanilla)
print(num_detected_events_ee)
print(num_detected_events_ee_abs_s)
# print(num_detected_events_lof)
# print(num_detected_events_ocsmv)

NameError: name 'ee_s' is not defined